In [4]:
# install surprise: conda install -c conda-forge scikit-surprise
import os
import numpy as np
import surprise
import pandas as pd
import surprise
from surprise import Reader, Dataset

In [4]:
rating_df = pd.read_csv("ml-25m/ratings.csv")
rating_df.head()

,userId,movieId,rating,timestamp
0,1,296,5.0,1147880044
1,1,306,3.5,1147868817
2,1,307,5.0,1147868828
3,1,665,5.0,1147878820
4,1,899,3.5,1147868510


In [244]:
train_df = rating_df[(rating_df['timestamp'] >= 1104505203) & (rating_df['timestamp'] <= 1230735592)]
test_df = rating_df[rating_df['timestamp'] >= 1230735600]

In [171]:
train_df = train_df.groupby(["userId", "movieId"])['rating'].sum().unstack().fillna(0)
train_df

movieId,1,2,3,4,5,6,7,8,9,10,...,64957,64969,64976,64983,64986,64990,64993,64997,65011,65025
userId,,,,,,,,,,,,,,,,,,,,,
1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,3.5,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
10,3.5,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
11,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
12,4.0,0.0,2.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
162511,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
162512,4.0,3.5,3.5,0.0,3.5,0.0,4.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
162513,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [286]:
# train, test를 나눈뒤 timestamp 제거
reader = surprise.Reader(rating_scale=(0.,5.))
data = surprise.Dataset.load_from_df(train_df, reader)
trainset = data

n_users = len(train_df["userId"].unqiue())
n_items = len(train_df["movieId"].unique())

AttributeError: 'Series' object has no attribute 'unqiue'

In [ ]:
class SVD_SGD(surprise.AlgoBase):
    
    def __init__(self, learning_rate, n_epochs, n_factors):
        
        self.lr = learning_rate  # learning rate for SGD
        self.n_epochs = n_epochs  # number of iterations of SGD
        self.n_factors = n_factors  # number of factors (몇 개의 Latent Factor로 요인 벡터를 만들지를 정함)
        
    def train(self, trainset):
        '''Learn the vectors p_u and q_i with SGD'''
        
        print('Fitting data with SGD...')
        
        # Randomly initialize the user and item factors.
        p = np.random.normal(0, .1, (trainset.n_users, self.n_factors))
        q = np.random.normal(0, .1, (trainset.n_items, self.n_factors))
        
        # SGD procedure
        for _ in range(self.n_epochs):
            for u, i, r_ui in trainset.all_ratings():
                err = r_ui - np.dot(p[u], q[i])
                # Update vectors p_u and q_i
                p[u] += self.lr * err * q[i]
                q[i] += self.lr * err * p[u]
                # Note: in the update of q_i, we should actually use the previous (non-updated) value of p_u.
                # In practice it makes almost no difference.
        
        self.p, self.q = p, q
        self.trainset = trainset

    def estimate(self, u, i):
        '''Return the estmimated rating of user u for item i.'''
        
        # return scalar product between p_u and q_i if user and item are known,
        # else return the average of all ratings
        if self.trainset.knows_user(u) and self.trainset.knows_item(i):
            return np.dot(self.p[u], self.q[i])
        else:
            return self.trainset.global_mean

In [287]:
algo = SVD_SGD(learning_rate=.01, n_epochs=10, n_factors=10)


AttributeError: 'DataFrame' object has no attribute 'trainset'

In [2]:
# 다시 함

In [5]:
train_df = rating_df[(rating_df['timestamp'] >= 1104505203) & (rating_df['timestamp'] <= 1230735592)]
test_df = rating_df[rating_df['timestamp'] >= 1230735600]

In [6]:
# timestamp 열 제거
del train_df["timestamp"]
del test_df["timestamp"]

In [8]:
train_df, test_df

(          userId  movieId  rating
 0              1      296     5.0
 1              1      306     3.5
 2              1      307     5.0
 3              1      665     5.0
 4              1      899     3.5
 ...          ...      ...     ...
 24997287  162521    56174     4.0
 24997298  162521    59315     4.5
 24997301  162521    59615     3.5
 24997302  162521    59725     4.0
 24997308  162521    60074     4.0
 
 [4602106 rows x 3 columns],
           userId  movieId  rating
 254            3        1     4.0
 255            3       29     4.5
 256            3       32     4.5
 257            3       50     5.0
 258            3      111     4.0
 ...          ...      ...     ...
 25000090  162541    50872     4.5
 25000091  162541    55768     2.5
 25000092  162541    56176     2.0
 25000093  162541    58559     4.0
 25000094  162541    63876     5.0
 
 [11472872 rows x 3 columns])

In [351]:
# # 데이터 변형
# def to_dict(frame):
#     if len(frame.columns) == 1:
#         if frame.values.size == 1: return frame.values[0][0]
#         return frame.values.squeeze()
#     grouped = frame.groupby(frame.columns[0])
#     d = {k: to_dict(g[:, 1:]) for k,g in grouped}
#     return d

In [9]:
reader = surprise.Reader(rating_scale=(1,5))
reader

In [10]:
col_list = ["userId", "movieId", "rating"]
data = surprise.Dataset.load_from_df(rating_df[col_list], reader)

In [13]:
print(col_list)
print(data)

['userId', 'movieId', 'rating']


In [14]:
trainset = data.build_full_trainset()
algo = surprise.SVD()
algo.fit(trainset)

In [16]:
testset = data.build_full_trainset()

In [17]:
testset = trainset.build_testset()
predictions = algo.test(testset)

In [18]:
predictions

[Prediction(uid=1, iid=296, r_ui=5.0, est=4.370697245593055, details={'was_impossible': False}),
 Prediction(uid=1, iid=306, r_ui=3.5, est=4.220443590687048, details={'was_impossible': False}),
 Prediction(uid=1, iid=307, r_ui=5.0, est=4.333543567062701, details={'was_impossible': False}),
 Prediction(uid=1, iid=665, r_ui=5.0, est=4.275862045906895, details={'was_impossible': False}),
 Prediction(uid=1, iid=899, r_ui=3.5, est=3.5924285195330445, details={'was_impossible': False}),
 Prediction(uid=1, iid=1088, r_ui=4.0, est=2.957287747058595, details={'was_impossible': False}),
 Prediction(uid=1, iid=1175, r_ui=3.5, est=4.0095802393501305, details={'was_impossible': False}),
 Prediction(uid=1, iid=1217, r_ui=3.5, est=3.8990074336439333, details={'was_impossible': False}),
 Prediction(uid=1, iid=1237, r_ui=5.0, est=3.9002736960369466, details={'was_impossible': False}),
 Prediction(uid=1, iid=1250, r_ui=4.0, est=3.9081503652590786, details={'was_impossible': False}),
 Prediction(uid=1, i

In [13]:
from surprise import accuracy
accuracy.rmse(predictions)

NameError: name 'predictions' is not defined

In [14]:
# 다시 함

In [ ]:
train_df = rating_df[(rating_df['timestamp'] >= 1104505203) & (rating_df['timestamp'] <= 1230735592)]
test_df = rating_df[rating_df['timestamp'] >= 1230735600]

In [15]:
# timestamp 열 제거
del train_df["timestamp"]
del test_df["timestamp"]

In [16]:
col_list = ["userId", "movieId", "rating"]

In [22]:
train_data = surprise.Dataset.load_from_df(train_df[col_list], reader)

trainset = train_data.build_full_trainset()

algo = surprise.SVD()
algo.fit(trainset)

In [23]:
test_data = surprise.Dataset.load_from_df(test_df[col_list], reader)
testset = test_data.build_full_trainset()

testset = testset.build_testset()
predictions = algo.test(testset)

predictions

from surprise import accuracy
accuracy.rmse(predictions)

RMSE: 0.9972


0.9971881464185708

In [24]:
predictions

[Prediction(uid=3, iid=1, r_ui=4.0, est=3.723732992615821, details={'was_impossible': False}),
 Prediction(uid=3, iid=29, r_ui=4.5, est=3.809212048896883, details={'was_impossible': False}),
 Prediction(uid=3, iid=32, r_ui=4.5, est=3.878753041228889, details={'was_impossible': False}),
 Prediction(uid=3, iid=50, r_ui=5.0, est=4.229518370446772, details={'was_impossible': False}),
 Prediction(uid=3, iid=111, r_ui=4.0, est=3.8895052553265854, details={'was_impossible': False}),
 Prediction(uid=3, iid=172, r_ui=4.0, est=2.7842314166253805, details={'was_impossible': False}),
 Prediction(uid=3, iid=173, r_ui=3.0, est=2.367250279174776, details={'was_impossible': False}),
 Prediction(uid=3, iid=214, r_ui=5.0, est=3.956388456304732, details={'was_impossible': False}),
 Prediction(uid=3, iid=260, r_ui=4.0, est=3.9245984930424527, details={'was_impossible': False}),
 Prediction(uid=3, iid=293, r_ui=5.0, est=4.129399117382281, details={'was_impossible': False}),
 Prediction(uid=3, iid=296, r_ui

In [26]:
# 이렇게 하면 의미가 없다 기존에 평가되어 있는 평점만 그대로 예측을 하는 것이다. 다른 방법이 필요하다.
# 피벗 테이블로 만들었을때 없는 값들이 예측이 되어야한다.
# NaN을 fillna(0)을 통해 채웠고 이 0으로 된 값들이 예측이 되어야한다.

In [37]:
# 다시 

In [4]:
# rating_df
rating_df = pd.read_csv("ml-25m/ratings.csv")

In [5]:
# movie_csv
movie_df = pd.read_csv("ml-25m/movies.csv")

In [6]:
del movie_df["genres"]  # 필요 없는 genres를 제거
movie_df

,movieId,title
0,1,Toy Story (1995)
1,2,Jumanji (1995)
2,3,Grumpier Old Men (1995)
3,4,Waiting to Exhale (1995)
4,5,Father of the Bride Part II (1995)
...,...,...
62418,209157,We (2018)
62419,209159,Window of the Soul (2001)
62420,209163,Bad Poems (2018)
62421,209169,A Girl Thing (2001)


In [7]:
merge_df = pd.merge(rating_df, movie_df, on="movieId", how="outer")

In [8]:
merge_df # 25003471

,userId,movieId,rating,timestamp,title
0,1.0,296,5.0,1.147880e+09,Pulp Fiction (1994)
1,3.0,296,5.0,1.439474e+09,Pulp Fiction (1994)
2,4.0,296,4.0,1.573939e+09,Pulp Fiction (1994)
3,5.0,296,4.0,8.307862e+08,Pulp Fiction (1994)
4,7.0,296,4.0,8.354447e+08,Pulp Fiction (1994)
...,...,...,...,...,...
25003466,NaN,208411,NaN,NaN,Eternal Blood (2002)
25003467,NaN,208413,NaN,NaN,Big Business (1929)
25003468,NaN,208415,NaN,NaN,The Student of Prague (1926)
25003469,NaN,208655,NaN,NaN,The Coldest Game (2019)


In [105]:
del merge_df["title"]
merge_df

,userId,movieId,rating,timestamp
0,1.0,296,5.0,1.147880e+09
1,3.0,296,5.0,1.439474e+09
2,4.0,296,4.0,1.573939e+09
3,5.0,296,4.0,8.307862e+08
4,7.0,296,4.0,8.354447e+08
...,...,...,...,...
25003466,NaN,208411,NaN,NaN
25003467,NaN,208413,NaN,NaN
25003468,NaN,208415,NaN,NaN
25003469,NaN,208655,NaN,NaN


In [9]:
# 결측치의 수가 같다.
merge_df.isnull().sum()

userId       3376
movieId         0
rating       3376
timestamp    3376
title           0
dtype: int64

In [10]:
merge_df = merge_df.dropna(0)
merge_df

,userId,movieId,rating,timestamp,title
0,1.0,296,5.0,1.147880e+09,Pulp Fiction (1994)
1,3.0,296,5.0,1.439474e+09,Pulp Fiction (1994)
2,4.0,296,4.0,1.573939e+09,Pulp Fiction (1994)
3,5.0,296,4.0,8.307862e+08,Pulp Fiction (1994)
4,7.0,296,4.0,8.354447e+08,Pulp Fiction (1994)
...,...,...,...,...,...
25000090,162358.0,200192,2.0,1.553453e+09,Den frusna leoparden (1986)
25000091,162358.0,200194,2.0,1.553454e+09,Tough Luck (2004)
25000092,162386.0,139970,3.5,1.549216e+09,I Don't Speak English (1995)
25000093,162386.0,200726,4.0,1.554651e+09,The Graduates (1995)


In [13]:
train_df = merge_df[(merge_df['timestamp'] >= 1104505203) & (merge_df['timestamp'] <= 1230735592)]
test_df = merge_df[merge_df['timestamp'] >= 1230735600]

In [14]:
reader = surprise.Reader(rating_scale=(1,5))
reader

In [15]:
# timestamp 열 제거
del train_df["timestamp"]
del test_df["timestamp"]

col_list = ["userId", "movieId", "rating"]

train_data = surprise.Dataset.load_from_df(train_df[col_list], reader)

trainset = train_data.build_full_trainset()

algo = surprise.SVD()
algo.fit(trainset)

KeyboardInterrupt: 

In [ ]:
test_data = surprise.Dataset.load_from_df(test_df[col_list], reader)
testset = test_data.build_full_trainset()

testset = testset.build_testset()
predictions = algo.test(testset)

In [72]:
from surprise import accuracy
accuracy.rmse(predictions)

RMSE: 0.9981


0.9980509279673461

In [17]:
## pivot table을 사용해야함 위 처럼 하게 되면 timestamp를 기준으로 데이터 프레임을 나누는 순간 다시 모든 영화들이 포함되지 않는 문제가 있음

In [20]:
# rating_df
rating_df = pd.read_csv("ml-25m/ratings.csv")

# movie_csv
movie_df = pd.read_csv("ml-25m/movies.csv")

del movie_df["genres"]  # 필요 없는 genres를 제거
movie_df

merge_df = pd.merge(rating_df, movie_df, on="movieId", how="outer")

merge_df # 25003471

del merge_df["title"]
merge_df

,userId,movieId,rating,timestamp
0,1.0,296,5.0,1.147880e+09
1,3.0,296,5.0,1.439474e+09
2,4.0,296,4.0,1.573939e+09
3,5.0,296,4.0,8.307862e+08
4,7.0,296,4.0,8.354447e+08
...,...,...,...,...
25003466,NaN,208411,NaN,NaN
25003467,NaN,208413,NaN,NaN
25003468,NaN,208415,NaN,NaN
25003469,NaN,208655,NaN,NaN


In [21]:
merge_df = merge_df.dropna(0)
merge_df

,userId,movieId,rating,timestamp
0,1.0,296,5.0,1.147880e+09
1,3.0,296,5.0,1.439474e+09
2,4.0,296,4.0,1.573939e+09
3,5.0,296,4.0,8.307862e+08
4,7.0,296,4.0,8.354447e+08
...,...,...,...,...
25000090,162358.0,200192,2.0,1.553453e+09
25000091,162358.0,200194,2.0,1.553454e+09
25000092,162386.0,139970,3.5,1.549216e+09
25000093,162386.0,200726,4.0,1.554651e+09


In [28]:
train_df = merge_df[(merge_df['timestamp'] >= 1104505203) & (merge_df['timestamp'] <= 1230735592)]
test_df = merge_df[merge_df['timestamp'] >= 1230735600]

In [24]:
test_df # userId, timestamp를 기억하여 전체 피벗에서 잘라 내야한다.

,userId,movieId,rating,timestamp
1,3.0,296,5.0,1.439474e+09
2,4.0,296,4.0,1.573939e+09
8,13.0,296,5.0,1.238030e+09
9,14.0,296,5.0,1.506209e+09
12,19.0,296,3.5,1.466675e+09
...,...,...,...,...
25000090,162358.0,200192,2.0,1.553453e+09
25000091,162358.0,200194,2.0,1.553454e+09
25000092,162386.0,139970,3.5,1.549216e+09
25000093,162386.0,200726,4.0,1.554651e+09


In [27]:
# # overflow 
# merge_table = merge_df.pivot(index="userId", columns="movieId", values="rating")